In [1]:
import sys, os, re

sys.path = (["../src/", "../"] if re.match(r"^(\w\:\\)|(/)", os.getcwd()) else []) + sys.path
import dataclasses
import qubx

import configparser
from pathlib import Path

%qubxd 

%load_ext autoreload
%autoreload 2

from qubx.core.interfaces import InstrumentsLookup, FeesLookup
from qubx.core.lookups import InstrumentsLookupFile, FeesLookupFile, InstrumentsLookupMongo 

from qubx.core.basics import Instrument, MarketType, TransactionCostsCalculator

from qubx.utils.misc import get_local_qubx_folder, load_qubx_resources_as_json, makedirs


⠀⠀⡰⡖⠒⠒⢒⢦⠀⠀   
⠀⢠⠃⠈⢆⣀⣎⣀⣱⡀  QUBX | Quantitative Backtesting Environment 
⠀⢳⠒⠒⡞⠚⡄⠀⡰⠁         (c) 2025, ver. 0.6.37
⠀⠀⠱⣜⣀⣀⣈⣦⠃⠀⠀⠀ 
        


In [67]:
parser = configparser.ConfigParser()
parser.read(Path(get_local_qubx_folder()) / "settings.ini")

['/home/quant0/.qubx/settings.ini']

In [75]:
# parser.get('instrument-lookup')

TypeError: RawConfigParser.get() missing 1 required positional argument: 'option'

In [76]:
dict(parser['instrument-lookup'])

{'type': 'mongo', 'mongo_url': 'mongodb://localhost:27017/'}

In [69]:
dict(parser['fees-lookup'])

{'type': 'file'}

In [2]:
class LookupsManager(InstrumentsLookup, FeesLookup):
    _i_lookup: InstrumentsLookup
    _t_lookup: FeesLookup

    def __new__(cls):
        if not hasattr(cls, 'instance'):
            cls.instance = super(LookupsManager, cls).__new__(cls)
            cls.instance._i_lookup = InstrumentsLookupFile()
            cls.instance._t_lookup = FeesLookupFile()

            # - load settings
            parser = configparser.ConfigParser()
            parser.read(Path(get_local_qubx_folder()) / "settings.ini")

            if 'instrument-lookup' in parser:
                cls.instance._i_lookup = LookupsManager.get_instrument_lookup(**dict(parser['instrument-lookup']))
            else:
                cls.instance._i_lookup = InstrumentsLookupFile()

            if 'fees-lookup' in parser:
                cls.instance._t_lookup = LookupsManager.get_fees_lookup(**dict(parser['fees-lookup']))
            else:
                cls.instance._t_lookup = FeesLookupFile()

        return cls.instance

    @staticmethod
    def get_instrument_lookup(type: str, **kwargs) -> InstrumentsLookup:
        match type.lower():
            case 'file':
                return InstrumentsLookupFile(**kwargs)
            case 'mongo':
                return InstrumentsLookupMongo(**kwargs)
            case _:
                raise ValueError(f"Invalid lookup type: {type}")

    @staticmethod
    def get_fees_lookup(type: str, **kwargs) -> FeesLookup:
        match type.lower():
            case 'file':
                return FeesLookupFile(**kwargs)
            case _:
                raise ValueError(f"Invalid lookup type: {type}")

    def find_symbol(self, exchange: str, symbol: str, market_type: MarketType | None = None) -> Instrument | None:
        return self._i_lookup.find_symbol(exchange, symbol, market_type)

    def find_instruments(
        self, exchange: str, quote: str | None = None, market_type: MarketType | None = None
    ) -> list[Instrument]:
        return self._i_lookup.find_instruments(exchange, quote, market_type)

    def find_aux_instrument_for(
        self, instrument: Instrument, base_currency: str, market_type: MarketType | None = None
    ) -> Instrument | None:
        return self._i_lookup.find_aux_instrument_for(instrument, base_currency, market_type)

    def find(self, exchange: str, spec: str | None) -> TransactionCostsCalculator:
        return self._t_lookup.find(exchange, spec)

In [101]:
lookup = LookupsManager()

In [ ]:
lookup._i_lookup

In [91]:
LookupsManager()

In [102]:
lookup.find_symbol("BINANCE.UM", "BTCUSDT")

BINANCE.UM:SWAP:BTCUSDT